In [3]:
import pandas as pd, numpy as np

# compare baseline fortified oil (lit) to baseline fortified oil (industry)

In [4]:
## pull in (previously final) input data sheet
outputs_dir = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/'
input_data_path = outputs_dir + 'lsff_input_coverage_data.csv'

input_data = pd.read_csv(input_data_path)

loc_id_map = {i:j for(i,j) in zip(input_data.location_name, input_data.location_id) }

In [5]:
# pull in (previously final) version of input data sheet with metadata
w_meta = pd.read_csv(outputs_dir + 'lsff_input_coverage_data_with_metadata.csv')

In [6]:
# pull in new numbers from Jonathan / industry
inputs_dir = "/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/inputs/"
new_data = pd.read_excel(inputs_dir + "20210411_LSFF_input_data_for_IHME.xlsx", header = [1,2,3])

In [7]:
# subset to the oil numbers we're interested in
new_data = new_data.set_index(new_data.columns[0])

new_oil = new_data['Baseline coverage']['Oil\n(percent of population eating specified oil)'][['Fortifiable oil.1','Compliance % of fortifiable']]

In [8]:
# pull columns we're interested in
rename_oil = {
    ('Location', 'Unnamed: 0_level_1', 'Unnamed: 0_level_2'): 'location_name',
 'Fortifiable oil.1': 'percent of population eating industrially produced vehicle',
 'Compliance % of fortifiable': 'percent of population eating fortified vehicle'
}

new_oil = new_oil.reset_index().rename(columns=rename_oil)

In [9]:
# reshape to our format
new_oil = pd.melt(new_oil, id_vars = 'location_name',
        value_vars = ['percent of population eating industrially produced vehicle',
       'percent of population eating fortified vehicle'],
       var_name='value_description',value_name='value_mean')

In [10]:
# fix c'ote divoire 
new_oil.loc[new_oil.location_name=="CÃ´te d'Ivoire","location_name"] = "Côte d'Ivoire"

In [11]:
# make sure we're not missing any locations / don't have any extra
assert(set(new_oil.location_name).symmetric_difference(set(input_data.location_name))==set())

In [12]:
# formatting
new_oil['vehicle'] = 'industry oil'
new_oil['nutrient'] = 'vitamin a'
new_oil['u5_applicable'] = True
new_oil['wra_applicable'] = True
new_oil['sub_population'] = np.NaN
new_oil['location_id'] = new_oil.location_name.map(loc_id_map)

In [13]:
# check formatting
new_oil.head()

,location_name,value_description,value_mean,vehicle,nutrient,u5_applicable,wra_applicable,sub_population,location_id
0,Angola,percent of population eating industrially prod...,72.320,industry oil,vitamin a,True,True,NaN,168
1,Bangladesh,percent of population eating industrially prod...,87.516,industry oil,vitamin a,True,True,NaN,161
2,Burkina Faso,percent of population eating industrially prod...,73.760,industry oil,vitamin a,True,True,NaN,201
3,Côte d'Ivoire,percent of population eating industrially prod...,72.000,industry oil,vitamin a,True,True,NaN,205
4,Cameroon,percent of population eating industrially prod...,72.000,industry oil,vitamin a,True,True,NaN,202


In [14]:
# from our (previously final) input sheet, select extracted rows of data (no regression or multiplicative results)
# then drop any rows duplicated for different nutrients
w_meta = w_meta[w_meta.estimation_status=='na'].drop(columns = 'nutrient').drop_duplicates()

In [15]:
# use ratio of scale over mean to estimate scale for new industry datapoints
w_meta['scale_over_mean'] = (w_meta.value_975_percentile - w_meta.value_025_percentile) / w_meta.value_mean
r = w_meta.loc[(w_meta.scale_over_mean!=np.inf) & (w_meta.scale_over_mean!=0),['vehicle','scale_over_mean']]

r_mean = r.scale_over_mean.mean()
r.loc[r.vehicle=="maize flour",'scale_over_mean'] = r_mean

r = r.groupby('vehicle').mean().dropna().rename(columns={'scale_over_mean':'r'}).reset_index()

In [16]:
r

,vehicle,r
0,maize flour,0.564982
1,oil,0.379944
2,wheat flour,0.688639


In [17]:
# we'll use the oil-specific ratio
new_oil['r'] = 0.379944

In [18]:
new_oil.head()

,location_name,value_description,value_mean,vehicle,nutrient,u5_applicable,wra_applicable,sub_population,location_id,r
0,Angola,percent of population eating industrially prod...,72.320,industry oil,vitamin a,True,True,NaN,168,0.379944
1,Bangladesh,percent of population eating industrially prod...,87.516,industry oil,vitamin a,True,True,NaN,161,0.379944
2,Burkina Faso,percent of population eating industrially prod...,73.760,industry oil,vitamin a,True,True,NaN,201,0.379944
3,Côte d'Ivoire,percent of population eating industrially prod...,72.000,industry oil,vitamin a,True,True,NaN,205,0.379944
4,Cameroon,percent of population eating industrially prod...,72.000,industry oil,vitamin a,True,True,NaN,202,0.379944


In [19]:
# input estimated CIs
new_oil['value_025_percentile'] = np.clip(new_oil.value_mean - (new_oil.r * new_oil.value_mean)/2, 0, 100)
new_oil['value_975_percentile'] = np.clip(new_oil.value_mean + (new_oil.r * new_oil.value_mean)/2, 0, 100)

In [20]:
output = input_data.append(new_oil)

/ihme/code/central_comp/miniconda_svc-ccomp/envs/v108/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [23]:
output.head()

,location_id,location_name,nutrient,r,sub_population,u5_applicable,value_025_percentile,value_975_percentile,value_description,value_mean,vehicle,wra_applicable
0,179,Ethiopia,na,NaN,NaN,True,24.950317,93.577380,percent of population eating vehicle,59.263848,maize flour,True
1,179,Ethiopia,na,NaN,NaN,True,11.437632,42.897396,percent of population eating industrially prod...,27.167514,maize flour,True
2,179,Ethiopia,folic acid,NaN,NaN,True,0.000000,0.000000,percent of population eating fortified vehicle,0.000000,maize flour,True
3,179,Ethiopia,iron,NaN,NaN,True,0.000000,0.000000,percent of population eating fortified vehicle,0.000000,maize flour,True
4,179,Ethiopia,vitamin a,NaN,NaN,True,0.000000,0.000000,percent of population eating fortified vehicle,0.000000,maize flour,True


In [28]:
compare_oil = pd.pivot_table(output.loc[(output.vehicle.isin(['oil','industry oil'])) & (output.value_description=="percent of population eating fortified vehicle")],
               index = ['location_name','nutrient'], columns = ['vehicle'], values = 'value_mean'
              )

In [34]:
compare_oil['diff'] = np.abs(compare_oil['industry oil'] - compare_oil['oil'])

In [35]:
compare_oil

,vehicle,industry oil,oil,diff
location_name,nutrient,,,
Angola,vitamin a,5.0,8.805705,3.805705
Bangladesh,vitamin a,42.8,42.849572,0.049572
Burkina Faso,vitamin a,23.0,51.083142,28.083142
Cameroon,vitamin a,20.0,30.496900,10.496900
China,vitamin a,5.0,NaN,NaN
Côte d'Ivoire,vitamin a,30.5,75.000000,44.500000
Democratic Republic of the Congo,vitamin a,3.0,0.000000,3.000000
Egypt,vitamin a,5.0,NaN,NaN
Ethiopia,vitamin a,5.0,0.000000,5.000000
